# Simple RAG pipeline using ollama
- Note: we will use `llama 3.1`
- Please install `ollama` first using installation steps in `http://localhost:11434/v1/`

In [1]:
import json
import torch
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm
from openai import OpenAI

## Build the elasticsearch index

In [2]:
# make sure have elasticsearch running (01-practice-rag-intro to see how to run it)
es_client = Elasticsearch("http://localhost:9200")
print(es_client.info())


{'name': '828d7d465fc1', 'cluster_name': 'docker-cluster', 'cluster_uuid': '1p66kkADQiG0ZkCayoUOQg', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [3]:
# build the index
index_name = "course-questions"

# load the raw document
with open('data/documents.json', 'rt') as f:
    docs_raw = json.load(f)

# create documents database (list of dict)
documents = []
for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

# index settings
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "question": {"type": "text"},
            "section": {"type": "text"},
            "course": {"type": "keyword"}
        }
    }
}

index_name = "course-questions"
es_client.indices.create(index=index_name, body=index_settings)

# index the documents
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [4]:
# function to search the index
def elastic_search(query):
    search_query = {
    "size": 5, 
    "query": {
        "bool": {  
            "must": {
                "multi_match": { 
                    "query": query, 
                    "fields": ["question^3", "text", "section"],
                    "type": "best_fields" 
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp" 
                }
            }
        }
    }
}
    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

elastic_search("How do I get the certificate?")

[{'text': "No, you can only get a certificate if you finish the course with a “live” cohort. We don't award certificates for the self-paced mode. The reason is you need to peer-review capstone(s) after submitting a project. You can only peer-review projects at the time the course is running.",
  'section': 'General course-related questions',
  'question': 'Certificate - Can I follow the course in a self-paced mode and get a certificate?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "All mage files are in your /home/src/folder where you saved your credentials.json so you should be able to access them locally. You will see a folder for ‘Pipelines’,  'data loaders', 'data transformers' & 'data exporters' - inside these will be the .py or .sql files for the blocks you created in your pipeline.\nRight click & ‘download’ the pipeline itself to your local machine (which gives you metadata, pycache and other files)\nAs above, download each .py/.sql file that corresponds to each block y

## Create ollama client

- To run ollama locally
```
ollama serve #in 1 terminal
ollama run llama3.1 # in another terminal
```

ollama sets  to `http://localhost:11434/v1/`

> Note, ollama starts the server on port 11434 automatically on mac, you don't need to do this if you already have ollama running
> See: https://github.com/ollama/ollama/issues/707



In [5]:
client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)


In [8]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
        model='llama3.1',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [10]:
query = 'I just disovered the course. Can I still join it?'
print(rag(query))

Yes, even if you don't register, you're still eligible to submit the homeworks.
